# Reading data

In [ ]:
import mne
import numpy as np
import pandas as pd
import re

## Load data

In [ ]:
def remove_unnecessary_events(raw, events, events_id, events_names = ['Time 0/', 'New Segment/']):
    events_to_remove = []
    for event_name in events_names:
        events_to_remove.append(events_id[event_name])
    
    # Filter out the unwanted events
    filtered_events = np.array([event.tolist() for event in events if event[2] not in events_to_remove])
    filtered_event_id = {key: value for key, value in events_id.items() if value not in events_to_remove}
    
    new_event_desc = {value:str(key) for key, value in filtered_event_id.items()}
    
    # Use mne.annotations_from_events to create new annotations
    new_annotations = mne.annotations_from_events(filtered_events, sfreq=raw.info['sfreq'], event_desc=new_event_desc)
    
    # Set new annotations to raw data
    raw_copy = raw.copy().set_annotations(new_annotations)

    return raw_copy

In [ ]:
signal_frequency = 64

In [ ]:
dir = '../data/sst_old'
filename = 'AD1406_SST14_Artif Rej 75'
# filename2 = 'AB2407_SST14_new_Artif Rej 75'
# dir = '../data/gng'
# filename = 'GNG_AA0303-64 el'

raw = mne.io.read_raw_brainvision(f'{dir}/{filename}.vhdr')
events, event_id = mne.events_from_annotations(raw)
raw = remove_unnecessary_events(raw, events, event_id)

events, event_id = mne.events_from_annotations(raw)

In [ ]:
# Define the mapping function using regular expressions
def map_event(event):
    # Patterns for matching
    patterns = {
        r'^Stimulus/ 3-B-NOSTOP.*': 'go/nostop',
        r'^Stimulus/ 3-B-STOP(\d+).*': lambda m: f'go/stop/{m.group(1)}',
        r'^Stimulus/ 3-R-B-NOSTOP.*': 'response/correct',
        r'^Stimulus/ 3-R-B-STOP(\d+).*': lambda m: f'response/incorrect/{m.group(1)}',
        r'^Stimulus/ 3-STOP(\d+).*': lambda m: f'stop/{m.group(1)}'
    }	
    # Check each pattern
    for pattern, replacement in patterns.items():
        match = re.fullmatch(pattern, event)
        if match:
            return replacement if not callable(replacement) else replacement(match)
    # Default return value if no pattern matches
    return 'unknown'

# Define the function to categorize events into 'go', 'response', or 'stop'
def categorize_type(event_general):
    if 'go' in event_general:
        return 'go'
    elif 'response' in event_general:
        return 'response'
    elif 'stop' in event_general:
        return 'stop'
    return 'unknown'

# Define the SSD mapping
ssd_mapping = {
    1: 100,
    2: 150,
    3: 200,
    4: 250,
    5: 300,
    6: 350,
    7: 400
}

# Define the function to map to SSD values
def map_ssd(event_general):
    if 'response/incorrect/' in event_general:
        match = re.search(r'response/incorrect/(\d+)', event_general)
        if match:
            number = int(match.group(1))
            return ssd_mapping.get(number, np.nan)  # Use np.nan for missing values
    return np.nan

In [ ]:
events_df = pd.DataFrame(events, columns=['latency', 'duration', 'id'])

# Invert the dictionary to map IDs to event names
id_to_event = {value: key for key, value in event_id.items()}

# Create a new 'event' column by mapping 'id' to event names
events_df['event'] = events_df['id'].map(id_to_event)
events_df['event_general'] = events_df['event'].apply(map_event)
events_df['type'] = events_df['event_general'].apply(categorize_type)
events_df['ssd'] = events_df['event_general'].apply(map_ssd)

# Calculate the mean of the 'ssd' column
ssd_mean = np.nanmean(events_df['ssd'])
# Create the 'ssd_centered' column by subtracting the mean from each 'ssd' value
events_df['ssd_centered'] = events_df['ssd'] - ssd_mean

In [ ]:
events_df

In [ ]:
events_df.to_csv(f'{filename}_events.csv')

In [ ]:
events_df[events_df['event_general'].str.contains('response/incorrect/')]

---

In [ ]:
f = f'{dir}/{filename}.dat'
f

In [ ]:
# n_data_ch = 1
# with open(f, "rb") as fid:
#     fid.seek(7)
#     block = np.empty((n_data_ch, 20))
#     for ii in range(20):
#         line = fid.readline().decode("ASCII")
#         line = line.strip()
#         # Not sure why we special-handle the "," character here,
#         # but let's just keep this for historical and backward-
#         # compat reasons  
#         if " " in line:
#             line_data = line.split()
#             print(line_data)
#         elif "," in line:
#             # likely exported from BrainVision Analyzer?
#             line_data = line.split(",")
#             print(line_data)
#         elif n_data_ch == 1:
#             line_data = [line]
#         else:
#             raise RuntimeError(
#                 "Unknown BrainVision data format encountered. "
#                 "Please contact the MNE-Python developers."
#             )
#         block[:n_data_ch, ii] = [float(part) for part in line_data]

In [ ]:
raw.info

In [ ]:
fig = raw.plot()

In [ ]:
tmin, tmax = -0.2, 1.6

# Read epochs
epochs = mne.Epochs(
    raw=raw,
    events=events,
    event_id=ids,
    tmin=tmin,
    tmax=tmax,
    baseline=None,
    reject_by_annotation=True,
    preload=True,
    event_repeated='merge',
    on_missing = "warn",
)

In [ ]:
fig = epochs.plot( n_epochs=3, events=True)

In [ ]:
events[:20,:]

In [ ]:
epochs.metadata